### Danh sách thư viện

In [5]:
%pip install bitcoinlib

Note: you may need to restart the kernel to use updated packages.


In [49]:
from bitcoinlib.keys import Key
from bitcoinlib.wallets import *
from bitcoinlib.values import *
from bitcoinlib.keys import HDKey
from bitcoinlib.services.services import Service
from bitcoinlib.transactions import Transaction, Output





## Create a P2PKH Script

### In ra màn hình thông tin của 1 ví

In [33]:
def print_info_wallet(wallet: Wallet, private_key):
    print("Infomation of Wallet")
    wallet.utxos_update()
    wallet.info()
    
    print("-----Infomation of Private key, Public key, Address------")
    print("Private_key: ", private_key)
    public_key = wallet.public_master().wif
    print("Public_key: ", public_key)
    print("Address: ", wallet.addresslist()[0])

### Sinh các khoá và Redeem Script rồi tạo ví (Chỉ chạy 1 lần đầu)

In [48]:
from bitcoinlib.wallets import Wallet
from bitcoinlib.keys import HDKey

# Thiết lập mạng và tạo các khóa
NETWORK = 'testnet'
k1 = HDKey(network=NETWORK)
k2 = HDKey(network=NETWORK)

# Tạo ví multisig 2-of-2
w1 = Wallet.create('multisig_2of2_1', sigs_required=2, keys=[k1, k2.public_master(multisig=True)], network=NETWORK)
w2 = Wallet.create('multisig_2of2_2', sigs_required=2, keys=[k1.public_master(multisig=True), k2], network=NETWORK)

# Lấy địa chỉ ví
address1 = w1.get_key().address

# Lưu địa chỉ ví và các khóa vào file txt
file_name = "Wallet.txt"
with open(file_name, "w") as file:
    file.write("Multisig Wallet Information\n")
    file.write("===========================\n\n")
    file.write(f"Cosigner  Address: {address1}\n")
    file.write("Keys and Signatures:\n")
    file.write("---------------------\n")
    file.write(f"Cosigner 1 Key (Private): {k1.private_hex}\n")
    file.write(f"Cosigner 1 Key (Public): {k1.public_hex}\n\n")
    file.write(f"Cosigner 2 Key (Private): {k2.private_hex}\n")
    file.write(f"Cosigner 2 Key (Public): {k2.public_hex}\n\n")

print(f"Wallet information has been saved to '{file_name}'.")
print(f"Deposit testnet Bitcoin to this address to create transaction: {address1}")


Wallet information has been saved to 'Wallet.txt'.
Deposit testnet Bitcoin to this address to create transaction: tb1qm0lqll5h5ryk2zmtpts38yrkueq2v4c6yuv85kfsmstcnu4g9kvsayngps


## Lock Funds

### Sử dụng Bitcoin testnet faucet để gửi BTC đến ví vừa tạo 

## Spend Locked Funds

### Tạo 1 ví mới để nhận 

- `wallet_name: `wallet_1_recieve
- `Private_key:` af237b45125eb4c63e0c7587e66d0d4e8f276086bb4e0046380b2d8ef63c98ec
- `Public_key:` vpub5ZgpxrojCsd8iKho7aDfDVPobmU2U4rLMLnEmUsWjEsEEA8vkjWHHDCPUZ64tfq7AjskpBMttwz94hf3eVHKsD34HwYG31Kn86zCrtnz4cx
- `Address:` tb1qs0rlmmq7dn0rh6csedpswv9rxfn743va44udlc

### Gửi coin

In [54]:
def spendLockFunds(wallet: Wallet, private_key_send: String, amount: float, destination_address: String):
    # wallet.info()
    wallet.utxos_update()
    amount = int(amount * 1e8)

    Tx = wallet.send_to(to_address=destination_address, amount=amount, network='testnet', priv_keys = private_key_send ,broadcast = True)
    Tx.info()
    
    print('Transaction finished! See your wallet status below:')
    wallet.utxos_update()
    # wallet.info()

In [ ]:

# Đường dẫn file chứa thông tin ví
import requests


file_name = "Wallet.txt"

try:
    if not os.path.exists(file_name):
        raise FileNotFoundError(f"File {file_name} không tồn tại.")

    with open(file_name, "r") as file:
        lines = file.readlines()

    if len(lines) < 8:
        raise ValueError("File không đúng cấu trúc hoặc thiếu thông tin.")

    # Trích xuất thông tin từ file
    cosigner_address = lines[0].split(": ")[1].strip()
    cosigner1_private_key = lines[3].split(": ")[1].strip()
    cosigner2_private_key = lines[6].split(": ")[1].strip()

    
    network = 'testnet'
    w1 = wallet_create_or_open('multisigcosigner1', sigs_required=2, keys=[k1, k2.public_master(multisig=True)], network=network)
    w2 = wallet_create_or_open('multisigcosigner2', sigs_required=2, keys=[k1.public_master(multisig=True), k2], network=network)

    print(" Địa chỉ multisig:", cosigner_address)
    print(" Cosigner 1 Private Key:", cosigner1_private_key)
    print(" Cosigner 2 Private Key:", cosigner2_private_key)

    # Kiểm tra thông tin private key
    if not cosigner1_private_key or not cosigner2_private_key:
        raise ValueError("Không tìm thấy khóa riêng (private key)")

    cosigner1_key = Key(import_key=cosigner1_private_key, network='testnet')
    cosigner2_key = Key(import_key=cosigner2_private_key, network='testnet')
    print(" Cosigner 1 Public Key:", cosigner1_key.public())
    print(" Cosigner 2 Public Key:", cosigner2_key.public())


    recipient_address = "tb1qflg0htt6ae3vcp0thc7rlrd9gelmv0e88q3frj"
    amount = 0.00001 
    print(" Địa chỉ nhận:", recipient_address)
    print(" Số tiền gửi (BTC):", amount)

    # Lấy UTXOs từ địa chỉ ví multisig
    print(" Đang lấy UTXO từ địa chỉ:", cosigner_address)
    service = Service(network='testnet')
    utxos = service.getutxos(address=cosigner_address)

    if not utxos:
        raise ValueError(f"Không có UTXO nào khả dụng tại địa chỉ: {cosigner_address}. Vui lòng nạp Bitcoin testnet.")

    print("Danh sách UTXOs:")
    total_balance = 0
    for utxo in utxos:
        print(f"  TxID: {utxo['txid']}, Output Index: {utxo['output_n']}, Value: {utxo['value']} Satoshis")
        total_balance += utxo['value']

    print(" Tổng số dư:", total_balance, "Satoshis")

    if total_balance < 0.000001 :
        raise ValueError("Số dư không đủ để thực hiện giao dịch. Vui lòng nạp thêm Bitcoin testnet.")

    # Tạo giao dịch
    transaction = Transaction(network='testnet')
    for utxo in utxos:
        txid = utxo['txid']
        output_n = utxo['output_n']
        transaction.add_input(txid, output_n)
        print(" Giao dịch trước khi ký:")
        print(transaction.info())
        
    
    # Ký giao dịch bằng cả hai cosigner
    
    transaction.sign([cosigner1_key,cosigner2_key])
    
    print("hehe")
    print(transaction.info())

    # In thông tin giao dịch sau khi ký
    # Phát giao dịch lên mạng Bitcoin testnet
    url = "https://blockstream.info/testnet/api/tx"
    
    # Gửi giao dịch đã ký đến API
    response = requests.post(url, data=transaction.as_hex())
    
    if response.status_code == 200:
        print("=================Giao dịch đã được gửi thành công.")
        print("TxID:", response.text)
    else:
        print("Lỗi khi gửi giao dịch:", response.text)

except FileNotFoundError as fnf_error:
    print(" Lỗi file:", fnf_error)

except ValueError as val_error:
    print(" Lỗi giá trị:", val_error)

except Exception as e:
    print(" Xảy ra lỗi:", e)


 Địa chỉ multisig: tb1qm0lqll5h5ryk2zmtpts38yrkueq2v4c6yuv85kfsmstcnu4g9kvsayngps
 Cosigner 1 Private Key: fd07f9b6ccbf0b2addf7e629df1fc099de09b09882a4f8d1e9fba2777686ada0
 Cosigner 2 Private Key: 538676ec41cc8a6f5c77cf28504cb7a7769ebba759221e75fcb1a3702215c6ef
 Cosigner 1 Public Key: 025aad3de42f83fc3a62d68f7ac2566c996de954e5f091110afafcb9bec9649f9a
 Cosigner 2 Public Key: 02a4a5e3a6fde7e021d45504698878bb58c1790f098edca15d4c929cedf6762c12
 Địa chỉ nhận: tb1qflg0htt6ae3vcp0thc7rlrd9gelmv0e88q3frj
 Số tiền gửi (BTC): 1e-05
 Đang lấy UTXO từ địa chỉ: tb1qm0lqll5h5ryk2zmtpts38yrkueq2v4c6yuv85kfsmstcnu4g9kvsayngps
Danh sách UTXOs:
  TxID: 692c7d5d5177fd9b683e3e343f40a02769506bdf9f32a260a56702b61107463d, Output Index: 1, Value: 10000 Satoshis
 Tổng số dư: 10000 Satoshis
 Giao dịch trước khi ký:
Transaction d21633ba23f70118185227be58a63527675641ad37967e2aa461559f577aec43
Date: None
Network: testnet
Version: 1
Witness type: segwit
Status: new
Verified: False
Inputs
-  0.00000000 tBTC 692c7d5d

### Kiểm tra đã nhận được coin chưa